# Using batch deployments for NLP processing

Importing the required libraries. This notebook requires:

- `azure-ai-ml`
- `mlflow`
- `azureml-mlflow`
- `numpy`
- `pandas`
- `huggingface`
- `torch`

In [153]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    AmlCompute,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## Accessing the Azure Machine Learning workspace

In [154]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## About the model

Let's review how the model is build. The model was built using TensorFlow along with the RestNet architecture ([Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)). This model has the following constrains that are important to keep in mind for deployment:

* In work with images of size 244x244 (tensors of `(224, 224, 3)`).
* It requires inputs to be scaled to the range `[0,1]`.

In [205]:
from transformers import pipeline, AutoTokenizer, TFBartForConditionalGeneration

summarizer = pipeline(
    "summarization", model="facebook/bart-large-cnn", tokenizer=tokenizer
)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Testing if the model works:

Let's save this model locally:

In [7]:
model_local_path = "bart-text-summarization/model"
summarizer.save_pretrained(model_local_path)

## Registering the model

We need to register the model in order to use it with Azure Machine Learning:

In [155]:
model_name = "bart-text-summarization"

In [156]:
if not any(filter(lambda m: m.name == model_name, ml_client.models.list())):
    print(f"Model {model_name} is not registered. Creating...")
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.CUSTOM_MODEL)
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Let's get a reference to the model:

In [216]:
model = ml_client.models.get(name=model_name, label="latest")

## Creating an scoring script to work with the model

In [210]:
%%writefile bart-text-summarization/code/transformer_scorer.py

import os
import numpy as np
from transformers import pipeline, AutoTokenizer, TFBartForConditionalGeneration
from datasets import load_dataset

def init():
    global model
    global tokenizer

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # load the model
    tokenizer = AutoTokenizer.from_pretrained(model_path, truncation=True, max_length=1024)
    model = TFBartForConditionalGeneration.from_pretrained(model_path)

def run(mini_batch):
    resultList = []

    ds = load_dataset('csv', data_files={ 'score': mini_batch})
    for text in ds['score']['text']:
        # perform inference
        input_ids = tokenizer.batch_encode_plus([text], truncation=True, padding=True, max_length=1024)['input_ids']
        summary_ids = model.generate(input_ids, max_length=130, min_length=30, do_sample=False)
        summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=False) for s in summary_ids]

        # Get results:
        resultList.append(summaries[0])

    return resultList

Overwriting bart-text-summarization/code/transformer_scorer.py


> tokenizer is configured to truncate the lenght of the text as the model has a limitation to 1024 tokens.

## Creating the deployment

First, let's create the endpoint that is going to host the batch deployments. Remember that each endpoint can host multiple deployments at any time, however, only one of them is the default one:

In [215]:
endpoint_name = "text-summarization-batch"
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="An batch service to perform text sumarization of content in CSV files",
)

In [164]:
ml_client.batch_endpoints.begin_create_or_update(endpoint)

Batch endpoints can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [236]:
compute_name = "cpu-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(
        name=compute_name, description="amlcompute", min_instances=0, max_instances=5
    )
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [160]:
from time import sleep

print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

Waiting for compute [DONE]


Let's create the environment. In our case, our model runs on `TensorFlow`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment

In [212]:
environment = Environment(
    conda_file="./bart-text-summarization/environment/conda.yml",
    image="mcr.microsoft.com/azureml/tensorflow-2.4-ubuntu18.04-py37-cpu-inference:latest",
)

Let's create a deployment under the given endpoint.

In [237]:
deployment = BatchDeployment(
    name="text-summarization-hfbart",
    description="A text summarization deployment implemented with HuggingFace and BART architecture",
    endpoint_name=endpoint.name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./bart-text-summarization/code/",
        scoring_script="transformer_scorer.py",
    ),
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size=1,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=3000),
    logging_level="info",
)

In [238]:
ml_client.batch_deployments.begin_create_or_update(deployment)

Let's update the default deployment name in the endpoint:

In [219]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint)

We can see the endpoint URL as follows:

In [220]:
endpoint.scoring_uri

'https://text-summarization-batch.eastus.inference.ml.azure.com/jobs'

## Testing the endpoint

Once the deployment is created, it is ready to recieve jobs. Let's first register a data asset so we can run the job against it. This data asset is a folder containing 1000 images from the original ImageNet dataset. We are going to download it first and then create the data asset:

In [232]:
data_path = "bart-text-summarization/data/"
dataset_name = "billsummary-small"

billsummary_data = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="A sample of the billsum dataset for text summarization, in CSV file format",
    name=dataset_name,
)

ml_client.data.create_or_update(billsummary_data)

Uploading data (14.95 MBs): 100%|██████████| 14946703/14946703 [00:00<00:00, 27100218.25it/s]




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'billsummary-small', 'description': 'A sample of the billsum dataset for text summarization, in CSV file format', 'tags': {}, 'properties': {}, 'id': '/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourceGroups/Analytics.Aml.Experiments.Workspaces/providers/Microsoft.MachineLearningServices/workspaces/aa-ml-aml-workspace/data/billsummary-small/versions/3', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/santiagxf-cpu/code/Users/fasantia/azureml-examples/sdk/python/endpoints/batch', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fa1226aadc0>, 'serialize': <msrest.serialization.Serializer object at 0x7fa12254f070>, 'version': '3', 'latest_version': None, 'path': 'azureml://subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourcegroups

In [233]:
billsummary_data = ml_client.data.get(name=dataset_name, label="latest")

Let's use this data as an input for the job:

In [239]:
input = Input(type=AssetTypes.URI_FOLDER, path=billsummary_data.id)

In [240]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, deployment_name=deployment.name, input=input
)

You can use the returned job object to check the status of the job:

In [241]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
text-summarization-batch,122bc442-91ac-4aa7-ab8f-c6f92c7a918e,pipeline,Preparing,Link to Azure Machine Learning studio


Notice how we are not indicating the deployment name in the invoke operation. That's because the endpoint automatically routes the job to the default deployment. Since our endpoint only has one deployment, then that one is the default one. You can target an specific deployment by indicating the argument/parameter `deployment_name`.

In [86]:
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name, endpoint_name=endpoint.name, input=input
)